# needed

# database with college football teams, colors, logos, game info (https://collegefootballdata.com/exporter)

# open ai key so I can use their image generation (https://www.geeksforgeeks.org/generate-images-with-openai-in-python/)

# s3 so I can save designs

 


In [1]:

from PIL import Image, ImageOps, ImageDraw, ImageFont
import warnings
import requests
import pandas as pd
import json
import datetime as dt
import cfbd
import helpers as hf
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import yaml
import random
import base64


In [2]:
base_config,shop_config = hf.get_config()


In [3]:
def monday_run():
    base_config = hf.get_config()
    cfbd_loader = hf.cfbp_handler(base_config['cfbd_api'])

    if hf.new_football_season():
        cfbd_loader.get_team_info()

    teams = hf.load_teams_table()
    # find a way to determine which ones are new
    cfbd_loader.get_schedule(2023)

    # this is a list of all the new shirts to design
    to_do = hf.to_do_list()
    for game in to_do:
        game_config = hf.parse_game(game,teams)
        config = hf.combine_configs(base_config,game_config)
        design = hf.build_design(config,test=False)

        hf.post(design)


- handle nan in teams table
- font auto sizing

In [4]:
#automate adding shirts to printfy/shopify

In [7]:

def post(post_dict,publish=False):
    upload_url = f"{post_dict['base_url']}/uploads/images.json"
    product_url = f"{post_dict['base_url']}/shops/{post_dict['shop_id']}/products.json"

    headers = {
    "Authorization": f"Bearer {post_dict['printify_access']}",
    "Content-Type": "application/json"
    }


    # this posts a design, but doesnt publish it

    with open(post_dict['path'], "rb") as img_file:
        img_b64 = base64.b64encode(img_file.read()).decode('utf-8')

    # Upload the image to the Printify media library
    data = {
    "file_name": post_dict['title'],
    "contents": img_b64
    }
    response = requests.post(upload_url, headers=headers, json=data)
    if response.status_code == 200:
        print('image sent to printify')
        image_id = response.json()["id"]
        variants = [{"id":id,"price":post_dict['price'],"is_enabled": True} for id in post_dict['variant_ids']]

        data = {
            "title": post_dict['title'],
            "description": post_dict['description'],
            "tags": post_dict['tags'].split(', '),  # Assuming tags are comma-separated in the CSV
            "blueprint_id": post_dict['blueprint_id'],
            "print_provider_id": post_dict['print_provider_id'],
            "variants": variants,
            "print_areas": [
                {
                    "variant_ids": post_dict['variant_ids'],  # Replace with the actual variant ID
                    "placeholders": [
                        {
                            "position": "back", # post to back of shirt, put a lil one on front with just date team score
                            "images": [
                                {
                                    "id": image_id,
                                    "x": 0.5,
                                    "y": 0.5,
                                    "scale": 1,
                                    "angle": 0
                                }
                            ]
                        }
                    ]
                }
            ]
        }

        response1 = requests.post(product_url, headers=headers, json=data)
        if response1.status_code == 200:
            print("Product posted successfully in Printify")
        else:
            print("Failed to post product in Printify")

        if publish:
            # this part does the publishing
            printify_publish = f"{post_dict['base_url']}/shops/{post_dict['shop_id']}/products/{json.loads(response1.text)['id']}/publish.json"

            update_data = {
                "title": True,
                "description": True,
                "images": True,
                "variants": True,
                "tags": True,
                "keyFeatures": True,
                "shipping_template": True
            }

            # Patch request to update the product status
            response2 = requests.post(printify_publish, headers=headers, json=update_data)
            if response2.status_code == 200:
                print("Product published successfully in Printify")
            else:
                print("Failed to publish product in Printify")
        else:
            print('unable to send image to printify')

In [8]:
post(shop_config,publish=True)

200
Product posted successfully in Printify
Product published successfully in Printify


In [ ]:
def create_collection(ids,week,year,post_dict):
    '''
    give this a list of product ids and it will add them to a collection for the week

    add something for the whole has the game happened yet aspect
    '''
    shopify_headers ={
    "X-Shopify-Access-Token": post_dict["shopify_access"],
    "Content-Type": "application/json",
    }
    collection_link = f"https://{post_dict['shop_name']}.myshopify.com/admin/api/2023-04/custom_collections.json"

    collects = [{"product_id":x} for x in ids]
    collection_data = {
    "custom_collection":{
        "title":f"Week {week}, {year}",
        "collects":collects
        }
    }
    response1 = requests.post(collection_link, headers=shopify_headers, json = collection_data)
    if response1.status_code==201:
        print("collection created")
    else:
        print("could not create collection")